In [1]:
import tensorflow as tf
import numpy as np
import random, os, time
import tensorflow.contrib.slim as slim  # 기존 tensorflow를 사용하기 쉽게 만들어 놓은 high-level API

from custom_op import conv2d, atrous_conv2d, relu, bn, max_pool

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
## --- 설정 변수 정의 ---
## 작업 디렉토리 정의
# 학습 데이터 파일명
train_list_file = "/src/hyebin/deeplab_v3/train_data_256.npy"
# 라벨 데이터 파일명
label_list_file = "/src/hyebin/deeplab_v3/train_label_256.npy"
# 분석 모델 디렉토리명
modeldir = "/src/hyebin/model/A-DEEPLAB-V3-CRACK-01"

## 학습 데이터 설정
# 학습할 데이터의 개수
training_count = 17    # (18~ OOM Error)

## 최적화 함수 설정
# 최적화 함수 학습률
learning_rate = 0.0001

## 학습 수행 설정
# 최대 학습 수행 횟수
epoch_count = 1000

In [3]:
## --- 학습 데이터 로딩 ---
# 데이터 구조
# train_list : 학습용 데이터(이미지 개수, 256, 256, 1)
# label_list : 학습용 라벨 데이터(이미지 개수, 256, 256, 1)
print("load train data: ", train_list_file) 
train_list = np.load(train_list_file, allow_pickle=True)

print("load train label data: ", label_list_file)
label_list = np.load(label_list_file, allow_pickle=True)

print(train_list.shape)
print(label_list.shape)

load train data:  /src/hyebin/deeplab_v3/train_data_256.npy
load train label data:  /src/hyebin/deeplab_v3/train_label_256.npy
(537, 256, 256, 1)
(537, 256, 256, 1)


In [4]:
## --- ResNet 모델 구성 ---

## 모든 convoltion 연산에 3x3 이하 크기의 커널 사용 
## feature map의 크기가 같은 layer는 출력 feature map 갯수가 동일함
## ResNet50 부터는 연산량을 줄이기 위해 Residual Block 내에 1x1, 3x3, 1x1 Convolution 연산을 쌓음
## 1x1 Convolution 연산으로 feature map의 갯수를 줄였다가, 3x3 연산을 거친 후,
## 1x1 Convolution 연산으로 차원을 늘려줌  (=병목 레이어)
    
## 차원이 같은 Residual(잔차) Block
## 입력값과 출력값의 차원이 같을 경우, 단순 add 연산만 진행함
def identity_block(inputs, filters, stage):
    filter1, filter2, filter3 = filters
    layer1 = relu(bn(conv2d(inputs, filter1, [1,1], name=stage+'_a_identity', padding='VALID')))
    layer2 = relu(bn(conv2d(layer1, filter2, [3,3], name=stage+'_b_identity')))
    layer3 = bn(conv2d(layer2, filter3, [1,1], name=stage+'_c_identity', padding='VALID'))
    layer4 = relu(tf.add(layer3, inputs))
    return layer4
    
    
## 차원이 다른 Residual(잔차) Block
## 입력값과 출력값의 차원이 다름
## 차원이 다르기 때문에 Conv_layer를 통해 projection shortcut
## projection shortcut : 입력값을 바로 더하지 않고 
##           1x1 convolution 연산을 stride를 설정하여 freature map의 크기와 개수를 맞춰준 후 더해줌
##           (즉, projection을 통해 차원을 맞춰주는 작업)
def conv_block(inputs, depths, stage, s=2):
    depth1, depth2, depth3 = depths
    layer1 = relu(bn(conv2d(inputs, depth1, [1,1], name=stage+'_a_conv', strides=[1, s, s, 1], padding='VALID')))
    layer2 = relu(bn(conv2d(layer1, depth2, [3,3], name=stage+'_b_conv')))
    layer3 = bn(conv2d(layer2, depth3, [1,1], name=stage+'_c_conv', padding='VALID'))
    shortcut = bn(conv2d(inputs, depth3, [1,1], name=stage+'_shortcut', strides=[1, s, s, 1], padding='VALID'))
    layer4 = relu(tf.add(layer3, shortcut))
    return layer4
    

## 입력값과 출력값의 차원이 같은 Atrous Convolution 구조의 Residual(잔차) Block
def atrous_identity_block(inputs, depths, stage, rate):
    depth1, depth2, depth3 = depths
    layer1 = relu(bn(atrous_conv2d(inputs, depth1, [1,1], rate, name=stage+'_a_identity')))
    layer2 = relu(bn(atrous_conv2d(layer1, depth2, [3,3], rate, name=stage+'_b_identity')))
    layer3 = bn(atrous_conv2d(layer2, depth3, [1,1], rate, name=stage+'_c_identity'))
    layer4 = relu(tf.add(layer3, inputs))
    return layer4
    
    
## 입력값과 출력값의 차원이 다른 Atrous Convolution 구조의 Residual(잔차) Block    
def atrous_conv_block(inputs, depths, stage, rate, s=2):
    depth1, depth2, depth3 = depths
    layer1 = relu(bn(atrous_conv2d(inputs, depth1, [1,1], rate, name=stage+'_a_conv')))
    layer2 = relu(bn(atrous_conv2d(layer1, depth2, [3,3], rate, name=stage+'_b_conv')))
    layer3 = bn(atrous_conv2d(layer2, depth3, [1,1], rate, name=stage+'_c_conv'))
    shortcut = bn(conv2d(inputs, depth3, [1,1], name=stage+'_shortcut', strides=[1, s, s, 1], padding='VALID'))
    layer4 = relu(tf.add(layer3, shortcut))
    return layer4

In [5]:
## --- 신경망 구성 ---
tf.reset_default_graph()

# --- 입력층 ---
x = tf.placeholder(dtype=tf.float32, shape=[None, 256, 256, 1], name="in")
y = tf.placeholder(dtype=tf.int64, shape=[None, 256, 256, 1], name="out")

## ResNet을 사용하여 특징 추출 (Encoder)
    
# ResNet50 : Convolution 연산 + Fully Connected Layer만 계산했을 때, 총 50개의 Layer (CNN)
# layer가 깊어질수록 성능이 더 좋아진다고 생각 (VGG-19보다 더 깊게 설계)
with tf.variable_scope('ResNet50') as scope:
    with tf.variable_scope('encoder') as scope:
        # strides : 필터를 적용하는 간격 (필터의 이동량)
        conv = conv2d(x, 64, [7,7], strides=[1,2,2,1], name='conv1')   # size 1/2
        # bn : tf.contrib.layers.batch_norm (Batch Normalization)
        # Batch Normalization : 신경망을 안정시켜주는 표준화 기법 
        conv = bn(conv)
        # relu : tf.nn.relu
        conv = relu(conv)
        # max_pool : Pooling Layer (tf.nn.max_pool)
        conv = max_pool(conv, ksize=[1,3,3,1], name='pool1')   # size 1/4
        print('conv : {}'.format(conv))
    
        conv = conv_block(conv, [64, 64, 256], '2_1', s=1)
        conv = identity_block(conv, [64, 64, 256], '2_2')
        conv = identity_block(conv, [64, 64, 256], '2_3')
        print('conv1 : {}'.format(conv))
    
        conv = conv_block(conv, [128, 128, 512], '3_1')
        conv = identity_block(conv, [128, 128, 512], '3_2')
        conv = identity_block(conv, [128, 128, 512], '3_3')
        print('conv2 : {}'.format(conv))
    
        conv = atrous_conv_block(conv, [256, 256, 1024], '4_1', 2, s=1)
        conv = atrous_identity_block(conv, [256, 256, 1024], '4_2',  2)
        conv = atrous_identity_block(conv, [256, 256, 1024], '4_3',  2)
        conv = atrous_identity_block(conv, [256, 256, 1024], '4_4',  2)
        conv = atrous_identity_block(conv, [256, 256, 1024], '4_5',  2)
        conv = atrous_identity_block(conv, [256, 256, 1024], '4_6',  2)
        print('conv3 : {}'.format(conv))
        
        conv = atrous_conv_block(conv, [512, 512, 2048], '5_1', 4, s=1)
        conv = atrous_identity_block(conv, [512, 512, 2048], '5_2', 4)
        conv = atrous_identity_block(conv, [512, 512, 2048], '5_3', 4)
        print('conv4 : {}'.format(conv))
    

    ## Atrous Pyrimid Pooling (Decoder)
        
    # ASPP : Atrous + Spatial Pyramid Pooling
    #      : rate를 다양하게 변환시켜 다양한 RF(Receptive Field)가 고려된 feature map을 생성할 수 있도록 함
    #      : 연산의 효율성 증가, 고정된 RF보다 다양하게 변화시킨 RF가 성능 향상 효과
    with tf.variable_scope('ASPP') as scope:
        feature_map_shape = conv.get_shape().as_list()
    
        # global average pooling
        # feature맵의 width, height 평균을 냄
        feature_map = tf.reduce_mean(conv, [1,2], keepdims=True)  # keepdims:차원 유지 여부
    
        feature_map = conv2d(feature_map, 256, [1,1], name='gap_feature_map')
        feature_map = tf.image.resize_bilinear(feature_map, [feature_map_shape[1], feature_map_shape[2]])
        print('\nfeature_map : {}'.format(feature_map))
     
        # rate : 필터의 픽셀 간의 거리를 나타내는 확장 비율
        rate1 = conv2d(conv, 256, [1,1], name='rate1')
        print('rate1 : {}'.format(rate1))
        rate6 = atrous_conv2d(conv, 256, [3,3], rate=6, name='rate6')
        print('rate6 : {}'.format(rate6))
        rate12 = atrous_conv2d(conv, 256, [3,3], rate=12, name='rate12')
        print('rate12 : {}'.format(rate12))
        rate18 = atrous_conv2d(conv, 256, [3,3], rate=18, name='rate18')
        print('rate18 : {}'.format(rate18))
        concated = tf.concat([feature_map, rate1, rate6, rate12, rate18], axis=3)  # concat:tensor 객체를 횡(가로) 방향으로 연결 (axis=3차원)
        print('concated : {}'.format(concated))

        net = conv2d(concated, 256, [1,1], name='net')
        print('net : {}'.format(net))
    
# logits:Neural Network의 최종 레이어가 내놓은 결과값
logits = conv2d(net, 2, [1,1], name='logits')
logits = tf.image.resize_bilinear(logits, size=[256, 256])
print('\nlogits : {}'.format(logits))
    
# 예측값
pred = tf.argmax(logits, axis=3)  # argmax:1차원 배열에서 가장 큰 값을 찾아 index return
pred = tf.expand_dims(pred, dim=3, name='pred')  # expand_dims:axis로 지정된 차원을 추가
print('\npred : {}'.format(pred))



conv : Tensor("ResNet50/encoder/pool1:0", shape=(?, 64, 64, 64), dtype=float32)
conv1 : Tensor("ResNet50/encoder/Relu_9:0", shape=(?, 64, 64, 256), dtype=float32)
conv2 : Tensor("ResNet50/encoder/Relu_18:0", shape=(?, 32, 32, 512), dtype=float32)
conv3 : Tensor("ResNet50/encoder/Relu_36:0", shape=(?, 32, 32, 1024), dtype=float32)
conv4 : Tensor("ResNet50/encoder/Relu_45:0", shape=(?, 32, 32, 2048), dtype=float32)

feature_map : Tensor("ResNet50/ASPP/ResizeBilinear:0", shape=(?, 32, 32, 256), dtype=float32)
rate1 : Tensor("ResNet50/ASPP/rate1/BiasAdd:0", shape=(?, 32, 32, 256), dtype=float32)
rate6 : Tensor("ResNet50/ASPP/rate6/BiasAdd:0", shape=(?, 32, 32, 256), dtype=float32)
rate12 : Tensor("ResNet50/ASPP/rate12/BiasAdd:0", shape=(?, 32, 32, 256), dtype=float32)
rate18 : Tensor("ResNet50/ASPP/rate18/BiasAdd:0", shape=(?, 32, 32, 256), dtype=float32)
concated : Tensor("ResNet50/ASPP/concat:0", shape=(?, 32, 32, 1280), dtype=float32)
net : Tensor("ResNet50/ASPP/net/BiasAdd:0", shape=

In [6]:
## --- loss 함수 정의 ---
# sparse_softmax_cross_entropy_with_logits:실측값인 label의 값은 인덱스의 숫자를 그대로 사용하고, 
#                                          예측 모델의 출력값은 인덱스의 one-hot encoding을 사용
loss = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=tf.squeeze(y, [3])))
print(logits)
print(y)
loss_summ = tf.summary.merge([tf.summary.scalar('loss', loss)])

print('loss : {}'.format(loss))
print('loss_summ : {}'.format(loss_summ))

## --- loss 함수의 최적화 함수 정의 ---
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

model_vars = tf.trainable_variables()
slim.model_analyzer.analyze_vars(model_vars, print_info=True)  # 현재 올라와있는 모든 트레이닝 변수명을 순서에 맞춰 출력해줌(scope)
                                                                # 변수 개수, 용량 등을 쉽게 볼 수 있음

Tensor("ResizeBilinear:0", shape=(?, 256, 256, 2), dtype=float32)
Tensor("out:0", shape=(?, 256, 256, 1), dtype=int64)
loss : Tensor("Mean:0", shape=(), dtype=float32)
loss_summ : Tensor("Merge/MergeSummary:0", shape=(), dtype=string)
---------
Variables: name (type shape) [size]
---------
ResNet50/encoder/conv1/conv_weight:0 (float32_ref 7x7x1x64) [3136, bytes: 12544]
ResNet50/encoder/conv1/conv_bias:0 (float32_ref 64) [64, bytes: 256]
ResNet50/encoder/BatchNorm/beta:0 (float32_ref 64) [64, bytes: 256]
ResNet50/encoder/BatchNorm/gamma:0 (float32_ref 64) [64, bytes: 256]
ResNet50/encoder/2_1_a_conv/conv_weight:0 (float32_ref 1x1x64x64) [4096, bytes: 16384]
ResNet50/encoder/2_1_a_conv/conv_bias:0 (float32_ref 64) [64, bytes: 256]
ResNet50/encoder/BatchNorm_1/beta:0 (float32_ref 64) [64, bytes: 256]
ResNet50/encoder/BatchNorm_1/gamma:0 (float32_ref 64) [64, bytes: 256]
ResNet50/encoder/2_1_b_conv/conv_weight:0 (float32_ref 3x3x64x64) [36864, bytes: 147456]
ResNet50/encoder/2_1_b_conv/con

(38781570, 155126280)

In [7]:
## --- 학습 수행 및  모델 저장 ---
with tf.Session() as sess:
    writer = tf.summary.FileWriter(modeldir + "/logs")
    writer.add_graph(sess.graph)
    
    # 신경망 가중치(weight) 초기화
    sess.run(tf.global_variables_initializer())
    
    total_batch = int(train_list.shape[0] / training_count)
    print('total_batch : {}\n'.format(total_batch))
    counter = 0
    
    # 전체 데이터에 대해 100번 학습 수행
    start_time = time.time()
    for epoch in range(epoch_count):
        total_loss = 0
        random.shuffle(train_list)  # 매 epoch마다 데이터셋 shuffling
        random.shuffle(label_list)
        
        for i in range(total_batch):
            # 학습용 데이터와 라벨
            # 랜덤 데이터를 training_count 만큼 가져옴
            train_x = train_list[i*training_count:i*training_count+training_count]
            train_y = label_list[i*training_count:i*training_count+training_count]
            
            _, _loss_summ, _loss = sess.run([optimizer, loss_summ, loss], feed_dict={x:train_x, y:train_y})
            writer.add_summary(summary = _loss_summ, global_step = counter)
            counter += 1
            total_loss += _loss
            
        print('Epoch:', '%03d' % (epoch + 1), ' Avg Loss: {:.6}\t'.format(_loss))

    end_time = time.time()
    print("running time: ", end_time - start_time)
    
    ## --- 학습 모델 저장 ---
    print("\nsave model\n")
    builder = tf.saved_model.builder.SavedModelBuilder(modeldir + "/model_12")
    signature = tf.saved_model.predict_signature_def(inputs={"in":x}, outputs={"out":pred})
    builder.add_meta_graph_and_variables(sess, tags=["ver1"], signature_def_map={"deep-crack-train-v1":signature})
    builder.save()
    print("\ncomplete")

total_batch : 31



KeyboardInterrupt: 